# Elmo Model 스팸메일 분류

참고 :

https://wikidocs.net/33930

http://hunterheidenreich.com/blog/elmo-word-vectors-in-keras/

### Tensorflow hub 설치 (Elmo Model Download)

In [1]:
!pip install tensorflow-hub

In [1]:
#tensorflow 1.x 버전 사용
%tensorflow_version 1.x

import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import keras

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
root_path = '/content/drive/My Drive/'
file_path = '/content/drive/My Drive/MachineLearning_project/sms-spam-collection-dataset/'

### Data load

In [7]:
data = pd.read_csv( file_path + 'spam.csv', encoding='latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
x_data = list(data['v2'])
y_data = list(data['v1'].replace(['ham','spam'],[0,1]))

print(len(x_data))
print(len(y_data))

5572
5572


### Train, Test 데이터 분류

In [9]:
train_len = int(len(x_data) * 0.8)
test_len = len(x_data) - train_len

print('train length : ', train_len)
print('test length : ', test_len)

train length :  4457
test length :  1115


In [10]:
x_train = np.array(x_data[:train_len])
y_train = np.array(y_data[:train_len])
x_test = np.array(x_data[train_len:])
y_test = np.array(y_data[train_len:])

print(x_train.shape)
print(x_test.shape)

(4457,)
(1115,)


### Build Model

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Lambda, Input

In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True) 

In [0]:
# Elmo 임베딩
def ElmoEmbedding(x) : 
  return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [35]:
input_layer = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ElmoEmbedding, output_shape=(1024, ))(input_layer)
x = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[input_layer], outputs=output_layer)
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=60)

Train on 4457 samples
4457/4457 [==============================] - 64s 14ms/sample - loss: 0.1076 - acc: 0.9594


### Evaluate Model

In [38]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

1115/1115 [==============================] - 18s 16ms/sample - loss: 0.6521 - acc: 0.6906

 테스트 정확도: 0.6906
